In [1]:
!lsusb

Bus 002 Device 001: ID 1d6b:0003 Linux Foundation 3.0 root hub
Bus 001 Device 006: ID 8087:07dc Intel Corp. 
Bus 001 Device 004: ID 1199:9041 Sierra Wireless, Inc. 
Bus 001 Device 009: ID 0b97:7772 O2 Micro, Inc. OZ776 CCID Smartcard Reader
Bus 001 Device 008: ID 0b97:7761 O2 Micro, Inc. Oz776 1.1 Hub
Bus 001 Device 007: ID 138a:0050 Validity Sensors, Inc. Swipe Fingerprint Sensor
Bus 001 Device 005: ID 04f2:b413 Chicony Electronics Co., Ltd 
Bus 001 Device 003: ID 04b4:6570 Cypress Semiconductor Corp. 
Bus 001 Device 002: ID 046d:c534 Logitech, Inc. Unifying Receiver
Bus 001 Device 030: ID 067b:2303 Prolific Technology, Inc. PL2303 Serial Port
Bus 001 Device 001: ID 1d6b:0002 Linux Foundation 2.0 root hub


In [2]:
!ls -l /dev/ttyUSB*

crw-rw-rw- 1 root dialout 188, 0 Mär  6 13:16 /dev/ttyUSB0


In [3]:
!cat /etc/udev/rules.d/55-usb_serial.rules

cat: /etc/udev/rules.d/55-usb_serial.rules: Datei oder Verzeichnis nicht gefunden


In [6]:
#  %pip install --user pyserial

  Using cached pyserial-3.5-py2.py3-none-any.whl (90 kB)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import serial

## read the configuration registers

In [6]:
ser = serial.Serial('/dev/ttyUSB0',timeout=0.5)  # open serial port
print(ser.name)         # check which port was really used
print(ser.timeout)
ser.write(b'\xC1\x00\x05')     # write a string
# read the response
response = ser.read(size=10)
print(response.hex())
ser.close()             # close port

/dev/ttyUSB0
0.5
c100050000640012


## write the configuration (persistent)

In [8]:
ser = serial.Serial('/dev/ttyUSB0',timeout=0.5)  # open serial port
print(ser.name)         # check which port was really used
print(ser.timeout)
# address = 00 00
# mode = 64 : 9600,8N1 air9.6k
# reg1 = 00
# freq = 12 : ch18=868.125 MHz
ser.write(b'\xC0\x00\x05\x00\x00\x64\x00\x12')     # command
# read the response
response = ser.read(size=10)
print(response.hex())
ser.close()             # close port

/dev/ttyUSB0
0.5
c100050000640012


## write to one module, read from the other
(pull M0,M1 low for normal transmission mode)

In [18]:
ser = serial.Serial('/dev/ttyUSB0',timeout=0.5)  # open serial port
print(ser.name)         # check which port was really used
print(ser.timeout)
ser.write(b'hello!')     # write a string
# read the response
response = ser.read(size=10)
print(response.hex(':'))
print(response)
ser.close()   

/dev/ttyUSB0
0.5
68:65:6c:6c:6f:21
b'hello!'


In [9]:
help(ser)

Help on Serial in module serial.serialposix object:

class Serial(serial.serialutil.SerialBase, PlatformSpecific)
 |  Serial(port=None, baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, write_timeout=None, dsrdtr=False, inter_byte_timeout=None, exclusive=None, **kwargs)
 |  
 |  Serial port class POSIX implementation. Serial port configuration is
 |  done with termios and fcntl. Runs on Linux and many other Un*x like
 |  systems.
 |  
 |  Method resolution order:
 |      Serial
 |      serial.serialutil.SerialBase
 |      io.RawIOBase
 |      _io._RawIOBase
 |      io.IOBase
 |      _io._IOBase
 |      PlatformSpecific
 |      PlatformSpecificBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  cancel_read(self)
 |  
 |  cancel_write(self)
 |  
 |  close(self)
 |      Close port
 |  
 |  fileno(self)
 |      For easier use of the serial port instance with select.
 |      WARNING: this function is not portable to different plat

## receive and print
(pull M0,M1 low for normal transmission mode)

In [20]:
ser = serial.Serial('/dev/ttyUSB0',timeout=0.3)  # open serial port
print(ser.name)         # check which port was really used
print(ser.timeout)

while True:
    # read the response
    response = ser.read(size=10)
    if len(response) > 0:
        print(response)
        

/dev/ttyUSB0
0.3
b'\xb9\x9d\xb9\r\nping.'
b'\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.'
b'\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'
b'ping.\r\n'


KeyboardInterrupt: 

In [21]:
ser.close()